# 🚀 Classification de Texte avec Hugging Face Transformers

Ce notebook présente un pipeline complet de **classification de texte** utilisant **DistilBERT** et la bibliothèque **Transformers** de Hugging Face.

## 📋 Table des Matières
1. [Installation des Dépendances](#dependencies)
2. [Chargement du Dataset AG News](#dataset)
3. [Justification du Choix du Modèle](#model-choice)
4. [Tokenisation des Données](#tokenization)
5. [Fine-tuning du Modèle](#training)
6. [Évaluation des Performances](#evaluation)
7. [Sauvegarde du Modèle](#save)
8. [Tests de Prédiction](#prediction)

## 🎯 Objectif
Développer un classifieur de texte capable de **catégoriser automatiquement des articles de presse** en 4 catégories : World, Sports, Business, et Sci/Tech.

## 📊 Résultats Attendus
- **🎯 Accuracy**: ~92% sur le jeu de test
- **📈 Dataset**: AG News (127,600 articles)
- **🏗️ Architecture**: DistilBERT fine-tuné
- **⚡ Temps d'entraînement**: ~10 epochs

---

## 1. Installation des Dépendances {#dependencies}


In [9]:
# Installation des bibliothèques Hugging Face et de scikit-learn pour l'évaluation
!pip install -q datasets transformers torch scikit-learn

print("✅ Bibliothèques installées.")

✅ Bibliothèques installées.


In [10]:
!pip install --upgrade datasets transformers huggingface-hub

## 2. Chargement du Dataset AG News {#dataset}

### 📰 À Propos d'AG News
Le dataset **AG News** est un benchmark populaire pour la classification de texte :

- **📊 Taille**: 127,600 articles de presse
- **📋 Classes**: 4 catégories d'actualités
- **🎯 Task**: Classification multi-classe
- **📈 Split**: 120K train / 7.6K test

### 🏷️ Classes Disponibles:
1. **World** (0) - Actualités mondiales
2. **Sports** (1) - Actualités sportives  
3. **Business** (2) - Actualités économiques
4. **Sci/Tech** (3) - Sciences et technologie


In [11]:
from datasets import load_dataset

# Chargement du jeu de données AG News
try:
    raw_datasets = load_dataset("ag_news")
    print("✅ Jeu de données 'ag_news' chargé avec succès.")
    print("\nStructure du jeu de données :")
    print(raw_datasets)

    # Affichage d'un exemple pour comprendre la structure
    print("\n--- Exemple de donnée (train) ---")
    example = raw_datasets['train'][0]
    print(example)

    # Les labels sont des entiers, créons un mappage pour la lisibilité
    label_names = raw_datasets['train'].features['label'].names
    print(f"\nLabels : {list(enumerate(label_names))}")
    print(f"Texte : '{example['text']}'")
    print(f"Label : {example['label']} ({label_names[example['label']]})")

except Exception as e:
    print(f"❌ Erreur lors du chargement du jeu de données : {e}")
    print("Il est possible que le service Hugging Face Hub soit temporairement indisponible.")

✅ Jeu de données 'ag_news' chargé avec succès.

Structure du jeu de données :
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

--- Exemple de donnée (train) ---
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}

Labels : [(0, 'World'), (1, 'Sports'), (2, 'Business'), (3, 'Sci/Tech')]
Texte : 'Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.'
Label : 2 (Business)


Justification du Choix du Modèle : DistilBERT
Pour cette tâche, nous allons choisir distilbert-base-uncased. C'est un choix stratégique pour plusieurs raisons :

Efficacité : DistilBERT est une version "distillée" de BERT. Il est environ 60% plus rapide et plus léger que BERT, tout en conservant plus de 95% de ses performances.
Adapté à Colab : Sa légèreté le rend idéal pour un fine-tuning sur des ressources limitées comme celles de Google Colab, permettant des itérations plus rapides.
Performance : Il offre un excellent compromis entre vitesse et précision pour les tâches de classification de texte.

## 4. Tokenisation des Données {#tokenization}

### 🔤 Processus de Tokenisation

La tokenisation transforme le texte brut en tokens numériques compréhensibles par le modèle :

#### 🔧 Étapes de Tokenisation:
1. **Segmentation**: Division en sous-mots (WordPiece)
2. **Mapping**: Conversion en IDs numériques
3. **Padding**: Uniformisation de la longueur
4. **Truncation**: Limitation à 512 tokens max

#### ⚙️ Configuration:
- **Tokenizer**: `distilbert-base-uncased`
- **Max Length**: 512 tokens
- **Padding**: Dynamique à la longueur max du batch
- **Truncation**: Activée pour les textes longs


In [12]:
from transformers import AutoTokenizer

# Chargement du tokenizer associé à DistilBERT
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Fonction pour tokeniser les textes
def tokenize_function(examples):
    # padding=True et truncation=True assurent que toutes les séquences ont la même longueur
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Appliquer la fonction de tokenisation à l'ensemble du jeu de données
# batched=True permet de traiter les données par lots pour plus de rapidité
print("🔄 Tokenisation des données en cours...")
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
print("✅ Données tokenisées.")

# On peut retirer la colonne "text" qui n'est plus nécessaire et renommer "label" en "labels"
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

# Création de sous-ensembles pour un entraînement plus rapide
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

🔄 Tokenisation des données en cours...


Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

✅ Données tokenisées.


## 5. Fine-tuning du Modèle {#training}

### 🎯 Configuration d'Entraînement

#### ⚙️ Hyperparamètres:
- **Learning Rate**: 2e-5 (recommandé pour BERT-family)
- **Batch Size**: 16 (train) / 16 (eval)
- **Epochs**: 10 (avec early stopping)
- **Weight Decay**: 0.01 (régularisation L2)
- **Optimizer**: AdamW (optimiseur par défaut)

#### 📊 Stratégie d'Entraînement:
1. **Subset Training**: 10K échantillons (entraînement rapide)
2. **Évaluation**: 1K échantillons de validation
3. **Monitoring**: Loss de validation à chaque epoch
4. **Sauvegarde**: Meilleur modèle automatiquement sauvé

### 🚀 Avantages du Fine-tuning:
- **Transfer Learning**: Exploitation des connaissances pré-acquises
- **Adaptation**: Spécialisation sur le domaine des actualités
- **Efficacité**: Moins de données requises qu'un entraînement from scratch


In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Chargement du modèle pré-entraîné, en spécifiant le nombre de classes (4 pour AG News)
# Assurez-vous que la variable 'model_checkpoint' est bien définie (ex: "distilbert-base-uncased")
# et que 'small_train_dataset' et 'small_eval_dataset' existent.

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4)

# Définition des arguments pour l'entraînement
training_args = TrainingArguments(
    output_dir="Pipeline-Classification-MLOps/models/huggingFace/results",

    # --- CORRECTION APPLIQUÉE ICI ---
    eval_strategy="epoch", # Remplacement de "evaluation_strategy" par "eval_strategy"
    # ---------------------------------

    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/TP/huginface/logs',
    report_to="none"
)

# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

# Lancement du fine-tuning
print("\n🚀 Début du fine-tuning...")
trainer.train()
print("✅ Fine-tuning terminé.")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🚀 Début du fine-tuning...


Epoch,Training Loss,Validation Loss
1,0.424600,0.284693
2,0.244600,0.296621
3,0.192500,0.311361
4,0.093200,0.360529
5,0.058100,0.393590
6,0.051200,0.447178
7,0.022300,0.479264
8,0.012600,0.521314
9,0.008100,0.528451
10,0.005100,0.537483


✅ Fine-tuning terminé.


## 6. Évaluation des Performances {#evaluation}

### 📊 Métriques d'Évaluation

Nous utilisons plusieurs métriques pour évaluer la performance de notre modèle :

#### 🎯 Métriques Principales:
- **Accuracy**: Pourcentage de prédictions correctes
- **Precision**: Exactitude des prédictions positives
- **Recall**: Capacité à identifier tous les cas positifs
- **F1-Score**: Moyenne harmonique de Precision et Recall

#### 📈 Résultats Attendus:
- **Accuracy globale**: ~92%
- **Performance par classe**: Équilibrée entre les 4 catégories
- **Meilleure classe**: Sports (textes distinctifs)
- **Plus difficile**: Business vs World (overlap possible)


In [14]:
import numpy as np
from sklearn.metrics import classification_report

print("\n📊 Évaluation des performances sur l'ensemble de test...")

# Obtenir les prédictions du modèle sur le jeu de test
predictions = trainer.predict(small_eval_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = small_eval_dataset["label"]

# Afficher le rapport de classification
print("\n--- Rapport de Classification ---")
print(classification_report(true_labels, predicted_labels, target_names=label_names))


📊 Évaluation des performances sur l'ensemble de test...



--- Rapport de Classification ---
              precision    recall  f1-score   support

       World       0.96      0.91      0.93       266
      Sports       0.98      0.99      0.98       246
    Business       0.89      0.87      0.88       246
    Sci/Tech       0.84      0.89      0.86       242

    accuracy                           0.92      1000
   macro avg       0.92      0.92      0.91      1000
weighted avg       0.92      0.92      0.92      1000



## 7. Sauvegarde du Modèle {#save}

### 💾 Stratégie de Sauvegarde

Le modèle et le tokenizer sont sauvegardés ensemble pour assurer la reproductibilité :

#### 🔧 Éléments Sauvegardés:
- **Modèle DistilBERT** fine-tuné (.bin)
- **Tokenizer** avec vocabulaire (.json)
- **Configuration** du modèle (.json)
- **Métadonnées** d'entraînement

#### 📁 Structure de Sauvegarde:
```
Pipeline-Classification-MLOps/models/huggingFace/
└── text-classifier-ag-news-distilbert/
    ├── config.json
    ├── pytorch_model.bin
    ├── tokenizer_config.json
    ├── tokenizer.json
    └── vocab.txt
```


In [15]:
# Définition du dossier de sauvegarde
output_model_dir = "Pipeline-Classification-MLOps/models/huggingFace/text-classifier-ag-news-distilbert"

# Sauvegarde du modèle et du tokenizer
trainer.save_model(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

print(f"\n✅ Modèle et tokenizer sauvegardés dans le dossier : '{output_model_dir}'")


✅ Modèle et tokenizer sauvegardés dans le dossier : '/content/drive/MyDrive/TP/huginface/text-classifier-ag-news-distilbert'


## 8. Tests de Prédiction en Production {#prediction}

### 🧪 Suite de Tests Complète

Cette section valide les performances du modèle sur différents types de textes :

#### 🎯 Types de Tests:

1. **Tests Simples** 🟢
   - Textes évidents pour chaque catégorie
   - Validation des cas d'usage basiques

2. **Tests Ambigus** 🟡
   - Articles avec éléments de plusieurs catégories
   - Test de la robustesse du modèle

3. **Tests Hors Distribution** 🔴
   - Textes non-journalistiques (cuisine, etc.)
   - Évaluation de la généralisation

#### 📊 Métriques de Confiance:
- **Très Haute** (>95%): Prédiction fiable
- **Haute** (80-95%): Prédiction probable
- **Modérée** (60-80%): Incertitude du modèle
- **Faible** (<60%): Prédiction peu fiable

### 🎯 Objectifs des Tests:
- Vérifier la **robustesse** du modèle
- Identifier les **cas limites**
- Tester la **généralisation** hors domaine


In [16]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- Configuration ---
# MODIFIEZ CE NOM si votre dossier de modèle est différent
MODEL_DIR = "Pipeline-Classification-MLOps/models/huggingFace/text-classifier-ag-news-distilbert"

# Les noms des classes dans l'ordre de leur index (0, 1, 2, 3)
CLASS_NAMES = ['World', 'Sports', 'Business', 'Sci/Tech']

# --- Phrases de test ---
test_sentences = {
    "Cas Simple (Sport)": "The team won the championship final after a thrilling match.",
    "Cas Simple (Business)": "The company's stock price surged after announcing record profits for the quarter.",
    "Cas Simple (Sci/Tech)": "Researchers have developed a new AI that can write compelling short stories.",
    "Cas Ambigu (Sport/Business/Tech)": "Formula 1 announced new engine regulations powered by sustainable fuels to reduce costs.",
    "Cas Hors Distribution (Cuisine)": "The recipe for this cake requires two cups of flour and a pinch of salt."
}


def predict(text, model, tokenizer):
    """
    Effectue une prédiction sur une seule chaîne de caractères.
    """
    # 1. Tokeniser le texte d'entrée
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # 2. Effectuer la prédiction
    with torch.no_grad():
        logits = model(**inputs).logits

    # 3. Interpréter les résultats
    # Appliquer Softmax pour convertir les logits en probabilités
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Obtenir la classe prédite et la confiance
    confidence, predicted_class_idx = torch.max(probabilities, dim=-1)
    predicted_class = CLASS_NAMES[predicted_class_idx.item()]

    return predicted_class, confidence.item()


def main():
    """
    Fonction principale pour charger le modèle et lancer les tests.
    """
    print(f"--- Début du test pour le modèle : {MODEL_DIR} ---")

    # 1. Vérifier si le dossier du modèle existe
    if not os.path.exists(MODEL_DIR):
        print(f"❌ Erreur : Le dossier du modèle '{MODEL_DIR}' n'a pas été trouvé.")
        return

    # 2. Charger le modèle et le tokenizer sauvegardés
    try:
        print("🔄 Chargement du modèle et du tokenizer...")
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
        tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
        print("✅ Modèle et tokenizer chargés avec succès.")
        # Mettre le modèle en mode évaluation
        model.eval()
    except Exception as e:
        print(f"❌ Erreur lors du chargement : {e}")
        return

    # 3. Lancer les tests sur les différentes phrases
    for test_name, sentence in test_sentences.items():
        print("-" * 40)
        print(f"🔬 Test en cours : {test_name}")
        print(f"   Texte : \"{sentence}\"")

        predicted_class, confidence = predict(sentence, model, tokenizer)

        print("\n--- RÉSULTAT DE LA PRÉDICTION ---")
        print(f"🎯 Classe prédite : {predicted_class}")
        print(f"💯 Confiance : {confidence:.2%}")
    print("-" * 40)


if __name__ == "__main__":
    main()

--- Début du test pour le modèle : /content/drive/MyDrive/TP/huginface/text-classifier-ag-news-distilbert ---
🔄 Chargement du modèle et du tokenizer...
✅ Modèle et tokenizer chargés avec succès.
----------------------------------------
🔬 Test en cours : Cas Simple (Sport)
   Texte : "The team won the championship final after a thrilling match."

--- RÉSULTAT DE LA PRÉDICTION ---
🎯 Classe prédite : Sports
💯 Confiance : 99.96%
----------------------------------------
🔬 Test en cours : Cas Simple (Business)
   Texte : "The company's stock price surged after announcing record profits for the quarter."

--- RÉSULTAT DE LA PRÉDICTION ---
🎯 Classe prédite : Business
💯 Confiance : 99.95%
----------------------------------------
🔬 Test en cours : Cas Simple (Sci/Tech)
   Texte : "Researchers have developed a new AI that can write compelling short stories."

--- RÉSULTAT DE LA PRÉDICTION ---
🎯 Classe prédite : Sci/Tech
💯 Confiance : 99.91%
----------------------------------------
🔬 Test en cours